In [9]:
import time
import math
import numpy as np
import matplotlib.pyplot as plt
with __import__('importnb').Notebook(): 
    import TargetRate_MarketSpeed_Helper
from scipy import interpolate
np.random.seed(30)
np.seterr(divide='ignore', invalid='ignore')
font = {'family': 'serif',
        'style': 'italic',
        # 'color': 'darkred',
        'weight': 1,
        'size': 16,
        }

In [10]:
# Plot Path Map with respective to Time
def PlotPath(t, T, Y, idxfig, title, lw=0.5, midline=None):
    fig_1 = plt.figure()
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)
    axes = fig_1.gca()
    axes.set_xlim([0, T])
    axes.set_ylim([np.nanmin(Y[idxfig]), np.nanmax(Y[idxfig])*1.1])

    for i in range(len(idxfig)):
        plt.plot(t, Y[idxfig[i]], linewidth=lw, label=i+1)
        
    if midline!=None:
        plt.plot(t, midline[1], '--k' )

    plt.ylabel(title,  fontdict=font)
    plt.xlabel('Time ($t$) ',  fontdict=font)
    plt.legend()
    plt.show()


In [11]:
# Plot Frequency Map 
def PlotHist(X, xlabel, bins=50, percentiles=[0.01, 0.1, 0.5, 0.9, 0.99]):
    # Visualizing price difference between strategies 
    fig_7 = plt.figure()
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    n, b, p = plt.hist(X, bins)

    q = np.quantile(X, percentiles)
    maxHeight = 1.1*np.max(n)
    for i in range(len(q)):
        plt.plot(q[i]*np.array([1,1]), np.array([0,maxHeight]), '--', label= percentiles[i])

    plt.ylabel('Frequency', fontdict=font)
    plt.xlabel(xlabel, fontdict=font)
    plt.ylim((0,maxHeight))
    plt.legend(('qtl=0.01','qtl=0.10','qtl=0.50','qtl=0.90','qtl=0.99'))
    plt.show()

In [12]:
#  Plot Heat/Density Map
def MakeHeatMap(t, y, xlabel, ylabel, fignum=False, nct=20, Nbins=100, Ndt=200, lower_threshold=0.1, upper_threshold=0.48):

    miny = np.nanmin(y)
    maxy = np.nanmax(y)
    dy = (maxy - miny) / Nbins
    bins = np.linspace(miny, maxy, 100)


    yr = np.full([Ndt, len(bins)], np.nan)

    mydt = (t[-1] - t[0]) / (Ndt - 1)

    tr = np.full([Ndt, ], np.nan)

    for i in range(Ndt):
        kk = np.where(t < t[0] + mydt * (i + 1))[-1][-1]
        count = np.histogram(y[:, kk], np.arange(miny, maxy + 0.00001, dy))
        yr[i, :] = count[0]
        tr[i] = t[kk].item()

    zr = yr.T / len(y[:, 0])

    if not fignum:
        fig = plt.figure()
    else:
        fig = plt.figure(fignum)
        
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)
    axes = fig.gca()
    axes.set_xlim(left=0)
    axes.set_ylim(bottom=0, top=np.max(maxy))
    x_cord_i, y_cord_i = np.meshgrid(tr, bins)
    zr[zr < np.max(zr)*lower_threshold] = 0
    zr[zr > np.max(zr)*upper_threshold] = np.max(zr)*upper_threshold
    cmap = plt.get_cmap('YlOrRd')

    plt.contour(x_cord_i, y_cord_i, zr, nct, cmap=cmap, levels=np.linspace(zr.min(), zr.max(), 1000))

    lim = np.around(np.max(zr), int(np.around(-(np.log(0.10972799999999999)/np.log(10)))))
    plt.colorbar(ticks=np.arange(0, np.max(zr), lim/10))

    y_1 = np.quantile(y, 0.05, axis=0)
    y_2 = np.quantile(y, 0.5, axis=0)
    y_3 = np.quantile(y, 0.95, axis=0)

    plt.plot(t, y_1, '--k', linewidth=2)
    plt.plot(t, y_2, '--k', linewidth=2)
    plt.plot(t, y_3, '--k', linewidth=2)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
